In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
FEATURES = 3
K = 10
label_name = "Eeg_label"

dataset_name = "ann_2022-03-01_2022-03-31.csv"
df_dataset = pd.read_csv(f"dataset/{dataset_name}", header=0, parse_dates=["timestamp"], index_col=0)

features = df_dataset.iloc[:,0:FEATURES].values
label = df_dataset.loc[:, label_name].values.reshape(-1,1)


# x_train = features[0:int(len(features)*0.7)]
# x_valid = features[int(len(features)*0.7):int(len(features)*0.8)]
# x_test = features[int(len(features)*0.8):]

# y_train = label[0:int(len(label)*0.7)]
# y_valid = label[int(len(label)*0.7):int(len(label)*0.8)]
# y_test = label[int(len(label)*0.8):]

In [7]:
fold_len = len(df_dataset)//10

In [12]:
for k in range(1,K):
    x_train = features[0:fold_len*k]
    x_test = features[fold_len*k:fold_len*(k+1)]
    y_train = label[0:fold_len*k]
    y_test = label[fold_len*k:fold_len*(k+1)]

    input_layer = tf.keras.layers.Input(shape=(FEATURES))
    ann_1 = tf.keras.layers.Dense(128, activation="relu")(input_layer)
    ann_2 = tf.keras.layers.Dense(64, activation="relu")(ann_1)
    ann_3 = tf.keras.layers.Dense(32, activation="relu")(ann_2)
    output_layer = tf.keras.layers.Dense(1, activation=None)(ann_3)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    loss = tf.keras.losses.MeanSquaredError()
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=loss, optimizer=opt)
    stopping_criterion = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=200,
                                                        verbose=1,
                                                        mode='auto')

    history = model.fit(x = x_train, y = y_train, epochs=1000, batch_size=32, validation_data=(x_test, y_test), callbacks=[stopping_criterion])
    model.save(f"models/{dataset_name.split('.')[0]}_fold_{k}.h5")

Epoch 1/1000
9/9 [==============================] - 1s 20ms/step - loss: 598.2578 - val_loss: 3497.6179
Epoch 2/1000
9/9 [==============================] - 0s 5ms/step - loss: 533.4598 - val_loss: 3252.4766
Epoch 3/1000
9/9 [==============================] - 0s 6ms/step - loss: 479.3160 - val_loss: 2974.5637
Epoch 4/1000
9/9 [==============================] - 0s 5ms/step - loss: 412.5917 - val_loss: 2656.1736
Epoch 5/1000
9/9 [==============================] - 0s 5ms/step - loss: 342.8581 - val_loss: 2253.5527
Epoch 6/1000
9/9 [==============================] - 0s 4ms/step - loss: 254.7672 - val_loss: 1824.9592
Epoch 7/1000
9/9 [==============================] - 0s 5ms/step - loss: 175.4752 - val_loss: 1349.8722
Epoch 8/1000
9/9 [==============================] - 0s 6ms/step - loss: 105.2822 - val_loss: 877.4783
Epoch 9/1000
9/9 [==============================] - 0s 5ms/step - loss: 41.0464 - val_loss: 496.8600
Epoch 10/1000
9/9 [==============================] - 0s 5ms/step - loss: 16

In [13]:
loss = {}
for k in range(1,K):
    x_test = features[fold_len*k:fold_len*(k+1)]
    y_test = label[fold_len*k:fold_len*(k+1)]
    model = tf.keras.models.load_model(f"models/{dataset_name.split('.')[0]}_fold_{k}.h5")
    loss[k] = model.evaluate(x_test, y_test)

9/9 [==============================] - 0s 999us/step - loss: 4442.2793


In [16]:
np.mean(list(loss.values()))

2284.093670315213

In [18]:
list(loss.values())

[272.6502685546875,
 417.6681213378906,
 147.94671630859375,
 39.09132385253906,
 680.2720336914062,
 5204.564453125,
 194.67062377929688,
 9157.7001953125,
 4442.279296875]